In [1]:
!nvidia-smi

Fri Apr  1 18:38:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   42C    P8    19W /  N/A |   1453MiB /  8192MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn

In [3]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [4]:
torch.cuda.device_count()

1

In [6]:
def try_gpu(i=0): #@save
    """如果存在，返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


def try_all_gpu(): #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [7]:
### 张量与gpu
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [8]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [9]:
# 因为只有一个gpu，所以返回的是cpu
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1844, 0.7179, 0.9655],
        [0.2401, 0.4215, 0.7673]])

In [11]:
Z = X.cuda()

In [12]:
Z

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [14]:
Y = torch.rand(2, 3, device=try_gpu())

In [15]:
Y + Z

tensor([[1.2147, 1.0160, 1.2096],
        [1.1265, 1.5416, 1.0164]], device='cuda:0')

In [16]:
net = nn.Sequential(nn.Linear(3, 1))

In [17]:
net = net.to(device=try_gpu())

In [18]:
net(X)

tensor([[-0.1951],
        [-0.1951]], device='cuda:0', grad_fn=<AddmmBackward>)

In [19]:
net[0].weight.data.device

device(type='cuda', index=0)